# Classic Search With Hyperspace
This notebook demonstrates the use of Hyperspace engine for classic (keyword and value matching) search. The datase set is taken from [Kaggle](https://www.kaggle.com/datasets/chicago/chicago-crime).

## The Dataset - Crimes In Chicago Dataset
From Kaggle:
"This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. Data is extracted from the Chicago Police Department's CLEAR (Citizen Law Enforcement Analysis and Reporting) system. In order to protect the privacy of crime victims, addresses are shown at the block level only and specific locations are not identified. This data includes unverified reports supplied to the Police Department. The preliminary crime classifications may be changed at a later date based upon additional investigation and there is always the possibility of mechanical or human error. Therefore, the Chicago Police Department does not guarantee (either expressed or implied) the accuracy, completeness, timeliness, or correct sequencing of the information and the information should not be used for comparison purposes over time."

The dataset includes 8 Million documents.


## Setting up the Hyper Space environment
Setting the enviorment requires the following steps


1. Download and install the client API
2. Connect to a server
3. Create data scheme file
4. Create collection
5. Ingest data
6. Run query

We mount a cloud folder which hosts the client files and install the client

###Install the Hyperspace client API


In [2]:
pip install git+https://github.com/hyper-space-io/hyperspace-py

  Cloning https://github.com/hyper-space-io/hyperspace-py to /tmp/pip-req-build-cym8wpgw
  Running command git clone --filter=blob:none --quiet https://github.com/hyper-space-io/hyperspace-py /tmp/pip-req-build-cym8wpgw
  Resolved https://github.com/hyper-space-io/hyperspace-py to commit e397b51e57fd6c3d83cdde8a8ed1b6b81d0509a7
  Preparing metadata (setup.py) ... done
  Created wheel for hyperspace: filename=hyperspace-1.0.0-py3-none-any.whl size=35778 sha256=63969fa2c89094433ae89b6ce77c9dead40af18360f06c3e38822779362324d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-dq_9vkzt/wheels/c4/96/59/f4b91d653fdbfc819e48a7dacbea1c9f3de59a1bc113aa840d
Successfully built hyperspace


###Connect to Server

Using the Hyperspace engine requires connection to a remote machine with pre-provided credendtials.

In [3]:
import hyperspace

hyperspace_client = hyperspace.HyperspaceClientApi(host='https://search-master-demo.development.hyper-space.xyz',
                                                   username=username, password=password)

We check the status before proceeding

In [5]:
cluster_status = hyperspace_client.cluster_status()
display(cluster_status)

[{'Collections size': {'all-MiniLM-L6-v2%20ArXiv%20titles': 376501},
  'FPGA memory usage in GB': '0.1045GB',
  'FPGA memory usage in percentage': '0.1045%',
  'Hostname': 'hyperspace-demo-0',
  'Number of total vectors': 376501},
 {'Number of data nodes': 1}]

###Create the data scheme file

Similarly to other search databases, Hyperspace requires a data schema file which outlines the data scheme.

In [6]:
import json

config = {
  "configuration": {
    "ID": {
      "type": "integer"
    },
    "Case Number": {
      "type": "keyword"
    },
    "Date": {
      "type": "date",
      "format": "MM/dd/yyyy hh:mm:ss a"
    },
    "Block": {
      "type": "keyword"
    },
    "IUCR": {
      "type": "keyword"
    },
    "Primary Type": {
      "type": "keyword"
    },
    "Description": {
      "type": "keyword"
    },
    "Location Description": {
      "type": "keyword"
    },
    "Arrest": {
      "type": "boolean"
    },
    "Domestic": {
      "type": "boolean"
    },
    "Beat": {
      "type": "integer"
    },
    "District": {
      "type": "integer"
    },
    "Ward": {
      "type": "integer"
    },
    "Community Area": {
      "type": "integer"
    },
    "FBI Code": {
      "type": "keyword"
    },
    "X Coordinate": {
      "type": "integer"
    },
    "Y Coordinate": {
      "type": "integer"
    },
    "Year": {
      "type": "integer"
    },
    "Updated On": {
      "type": "date",
      "format": "MM/dd/yyyy hh:mm:ss a"
    },
    "Latitude": {
      "type": "float"
    },
    "Longitude": {
      "type": "float"
    },
    "Location": {
      "type": "geo_point",
      "struct_type": "list"
    }
  }
}

with open('crime-config.json', 'w') as f:
    f.write(json.dumps(config, indent=2))



The dataset fields


1. **Case Number {'type': 'keyword'}** -
The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.
2. **Date {'type': 'date', 'format': 'MM/dd/yyyy hh:mm:ss a'}** - Date when the incident occurred. this is sometimes a best estimate.
3. **Block {'type 'keyword'}** -The partially redacted address where the incident occurred, placing it on the same block as the actual address.
4. **IUCR {'type 'keyword'}** - The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.
5. **Primary Type {'type 'keyword'}** - The primary description of the IUCR code.
6. **Description {'type 'keyword'}** - The secondary description of the IUCR code, a subcategory of the primary description.
7. **Location Description {'type 'keyword'}** - Description of the location where the incident occurred.
8. **Arrest {'type 'boolean'}** - Indicates whether an arrest was made.
9. **Domestic {'type 'boolean'}** - Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.
10. **Beat {'type 'integer'}** - Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74.
11. **District {'type 'integer'}** - Indicates the police district where the incident occurred. See the districts at https://data.cityofchicago.org/d/fthy-xz3r.
12. **Ward {'type 'integer'}** - The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76.
13. **Community Area {'type 'integer'}** - Indicates the community area where the incident occurred. Chicago has 77 community areas. See the community areas at https://data.cityofchicago.org/d/cauq-8yn6.
14. **FBI Code {'type 'keyword'}** - Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html.
15. **X Coordinate {'type 'integer'}** - The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.
16. **Y Coordinate {'type 'integer'}** - The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.
17. **Year {'type 'integer'}** - Year the incident occurred.
18. **Updated On {'type 'date', 'format 'MM/dd/yyyy hh:mm:ss a'}** - Date and time the record was last updated.
19. **Latitude {'type 'float'}** - The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.
20. **Longitude {'type 'float'}** - The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.
21. **Location {'type 'geo_point', 'struct_type 'list'}** - The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block.

  
  
  
  
  



### Create Collection
Collections are used to store data of similar context, etc. We will erase the current collection, create a new one, and ingest data.

In [8]:
delete_collections = True
collection_name = 'CrimesInChicago'

if delete_collections:
  if collection_name in cluster_status[0]['Collections size']:
    hyperspace_client.delete_collection(collection_name)

hyperspace_client.create_collection('crime-config.json', collection_name)
hyperspace_client.cluster_status()

[{'Collections size': {'CrimesInChicago': 0,
   'all-MiniLM-L6-v2%20ArXiv%20titles': 376501},
  'FPGA memory usage in GB': '0.1047GB',
  'FPGA memory usage in percentage': '0.1047%',
  'Hostname': 'hyperspace-demo-0',
  'Number of total vectors': 376501},
 {'Number of data nodes': 1}]

### Ingest data

We load the datasets from and ingest it in batches of 500 documents

In [9]:
metadata = open('crimes-dataset-processed_data.json')

In [ ]:
BATCH_SIZE = 500

batch = []
for i, metadata_row in enumerate(metadata):
    row = {key: value for key, value in json.loads(metadata_row).items() if key in config["configuration"].keys()}
    batch.append(hyperspace.Document(str(i), row))

    if i % BATCH_SIZE == 0:
        response = hyperspace_client.add_batch(batch, collection_name)
        batch.clear()
        print(i, response)



In [13]:
hyperspace_client.commit(collection_name)

{'code': 200, 'message': 'Dataset committed successfully', 'status': 'OK'}

#Define Logic and Run a Query
We will build a classic search query using Hyperspace. In the query,  we will randomly select an incident and find similar one. The score function is given under
Crime_Score_Function.py


In [14]:
from pprint import pprint

input_vector = hyperspace_client.get_document(collection_name, "65")
pprint(input_vector)

{'Arrest': False,
 'Beat': 522,
 'Block': '0000X W 114TH PL',
 'Case Number': 'HY411431',
 'Community Area': 49,
 'Date': 1441439580,
 'Description': 'AGGRAVATED:KNIFE/CUTTING INSTR',
 'District': 5,
 'Domestic': False,
 'FBI Code': '04B',
 'ID': 10224815,
 'IUCR': '0420',
 'Latitude': 41.686218054,
 'Location': [41.686218054, -87.623739069],
 'Location Description': 'RESIDENCE',
 'Longitude': -87.623739069,
 'Primary Type': 'BATTERY',
 'Updated On': 1518270601,
 'Ward': 34,
 'X Coordinate': 1178072,
 'Y Coordinate': 1829048,
 'Year': 2015}


We will use a very simple logic, which matchs the description and location, and make sure case number doesn't match so we won't get back the same result.

We use the following logic:


*   Match crime description and not case number
*   geo_dist match (ge distance)
*   Match district and window_match the date
*   Match Block

Score function can be view in the next block





In [ ]:
def similarity_sf(Q, V):
    score = 0.0

    if match('Description') and not match('Case Number'):
        score = rarity_max('Description')
    if geo_dist_match('Location',4):
        score += 10
    if match('District') and window_match('Date', "100d", "40d"):
        score -= 5
    return score


In [22]:
response = hyperspace_client.set_function('Crime_Score_Function.py', collection_name=collection_name, function_name='similarity_sf')

query = {
    'params': input_vector,
     "query": {"boost": 1}
}

results = hyperspace_client.search(query,
                                        size=30,
                                        function_name='similarity_sf',
                                        collection_name=collection_name)
print("query run time:", results["took_ms"])
for i, result in enumerate(results['similarity']):
  vector_api_response = hyperspace_client.get_document(document_id=result['document_id'], collection_name=collection_name)
  print(i + 1, "id", result['document_id'], "score = " , result["score"] ,":", vector_api_response)



query run time: 3.16817
1 id 125277 score =  14.190526962280273 : {'Arrest': False, 'Beat': 522, 'Block': '0000X W 114TH PL', 'Case Number': 'HY554174', 'Community Area': 49, 'Date': 1451404800, 'Description': 'AUTOMOBILE', 'District': 5, 'Domestic': False, 'FBI Code': '07', 'ID': 10362444, 'IUCR': '0910', 'Latitude': 41.686211437, 'Location': [41.686211437, -87.624445722], 'Location Description': 'STREET', 'Longitude': -87.624445722, 'Primary Type': 'MOTOR VEHICLE THEFT', 'Updated On': 1452176074, 'Ward': 34, 'X Coordinate': 1177879, 'Y Coordinate': 1829044, 'Year': 2015}
2 id 13936 score =  14.190526962280273 : {'Arrest': True, 'Beat': 522, 'Block': '0000X W 114TH PL', 'Case Number': 'HY434421', 'Community Area': 49, 'Date': 1442949840, 'Description': 'AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON', 'District': 5, 'Domestic': True, 'FBI Code': '04B', 'ID': 10246412, 'IUCR': '0497', 'Latitude': 41.68621653, 'Location': [41.68621653, -87.623936782], 'Location Description': 'RESIDENCE'

We display the top 30 results. Note that results with similar score are ordered arbitrarily, so more complex logic will likely result in better outcome.

For more information, visit us at [Hyperspace](https://www.hyper-space.io/)